<a href="https://colab.research.google.com/github/newstage75/python_data_analysis_100knock/blob/main/%E7%AC%AC5%E7%AB%A0/%E3%80%90%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%80%91%E9%A1%A7%E5%AE%A2%E3%81%AE%E9%80%80%E4%BC%9A%E3%82%92%E4%BA%88%E6%B8%AC%E3%81%99%E3%82%8B10%E6%9C%AC%E3%83%8E%E3%83%83%E3%82%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 作業フォルダへの移動を行います。
# 人によって作業場所が異なるので、その場合作業場所を変更してください。（ここでは、第４章のデータを引き続き使用）
import os
os.chdir('/content/drive/MyDrive/データサイエンス/20240505python実践データ分析100本ノック/実習用Myフォルダ/第４章')

In [5]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('uselog_months.csv')

In [6]:
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


In [8]:
year_months = list(uselog_months["年月"].unique())
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [9]:
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
  tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]].copy()
  tmp.rename(columns={"count":"count_0"}, inplace=True)
  tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]].copy()
  del tmp_before["年月"]
  tmp_before.rename(columns={"count":"count_1"}, inplace=True)
  tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
  uselog = pd.concat([uselog,tmp], ignore_index=True)
uselog.head()


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


上記のuselogでは当月と１ヶ月前の利用回数を集計している。

## ノック42:退会前月の退会顧客データを作成

退会の予測をする目的は"退会を未然に防ぐこと"なので、退会前月のデータから退会を予測する。

In [10]:
# 退会した顧客に絞り込んで、end_date列の1ヶ月前の年月を取得し、ノック41で整形したuselogとcustomer_id、年月をキーにして結合する。
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flag,calc_date,membership_period
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23


In [11]:
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"]) #date_time型に変換

<ipython-input-11-9558bac5c90f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"] = None
<ipython-input-11-9558bac5c90f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"]) #date_time型に変換
